In [1]:
import sys 
#sys.path.append("codes/modules") # add custom Vibe 's modules
sys.path.append("../..") # add standard 's modules
sys.path.append('../modules')

import pyfesom as pf
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.basemap import Basemap
import numpy as np
from netCDF4 import Dataset
import os
import time

No joblib
no cmocean


In [2]:
# ==============================================================================
# Running this file loads tracers from old FESOM-REcoM2 output file (oce.mean.nc)
# and saves each tracer in an individual file
# 
#  Input:
#  - mesh_id: Name of mesh, will be added to the netcdf name
#  - meshpath: Speciefies where the target mesh is stored
#  - save_netcdf: If true, netcdf will be created
#  - delete_old_netcdf: If a netcdf file with the same name exists, a new cannot 
#    be made. If set to true, an old netcdf with the same name will be deleted
#
#  Output:
#  - netcdf file for each tracer in the old file
#  
#  During running, keep an eye on the output in the terminal, to see if it 
#  makes sense. 
#
# ==============================================================================

In [3]:
# Loading mesh for run

mesh_id    = 'meshArc4.5'
meshpath   = '/home/hbkoziel/pyfesom/data/mesh/'+mesh_id+'/'            # Defining path where mesh is stored
mesh = pf.load_mesh(meshpath, usepickle=True, get3d=True)                                    # Loading mesh, stores it in mesh.****  
#mesh = pf.fesom_mesh(meshpath, get3d=True)
#mesh.zlevs = -mesh.zlevs                                            # Depth is made negative

tracername = 'w50m'
first_year = 1985
last_year  = 2015
years      = np.arange(first_year,last_year+1,1)
runid	= 'Arc12'

/home/hbkoziel/pyfesom/data/mesh/meshArc4.5/pickle_mesh
2
The usepickle == True)
The pickle file for python 2 exists.
The mesh will be loaded from /home/hbkoziel/pyfesom/data/mesh/meshArc4.5/pickle_mesh


In [4]:
# ==============================================================================
# Settings for netcdf file

save_netcdf       = True                                            # Saves the interpolated field in netcdf file
delete_old_netcdf = True                                            # If a netcdf file with the same name exists it will be deleted
input_directory  = '/scratch/usr/hbkvsk12/hlrn3_work2/results/' # Where the netcdf is saved
output_directory  = '/scratch/usr/hbkoziel/'+runid+'/netcdf/'
plot_netcdf       = True                                           # Reads DIN from the created netcdf file, else it plots the interpolated field (should be the same)

In [5]:
depth = 50

In [7]:
ind_mld = np.max(np.argwhere(mesh.zlevs == depth)) # depth above MLD
ind_depth = np.array(mesh.n32[:,ind_mld]) # depth max 580m

for ind in range(0,len(years)):
    netcdf_name       = tracername+'.'+str(years[ind])+'.monthly.nc'

    # ==============================================================================
    # Loading data

    ncfile	= input_directory+runid+'/Oldfiles/'+runid+'.'+str(years[ind])+'.oce.mean.nc'
    f	= Dataset(ncfile,'r')

    if years[ind] >= 2000:
        t1 = f.variables['w'][0:15,ind_depth].mean(axis=0)
        t2 = f.variables['w'][15:29,ind_depth].mean(axis=0)
        t3 = f.variables['w'][29:45,ind_depth].mean(axis=0)
        t4 = f.variables['w'][45:60,ind_depth].mean(axis=0)
        t5 = f.variables['w'][60:75,ind_depth].mean(axis=0)
        t6 = f.variables['w'][75:90,ind_depth].mean(axis=0)
        t7 = f.variables['w'][90:106,ind_depth].mean(axis=0)
        t8 = f.variables['w'][106:121,ind_depth].mean(axis=0)
        t9 = f.variables['w'][121:136,ind_depth].mean(axis=0)
        t10 = f.variables['w'][136:152,ind_depth].mean(axis=0)
        t11 = f.variables['w'][152:167,ind_depth].mean(axis=0)
        t12 = f.variables['w'][166:,ind_depth].mean(axis=0)
        tracer = [t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12]
    else:
        t1 = f.variables['w'][0,ind_depth]
        t2 = f.variables['w'][1,ind_depth]
        t3 = f.variables['w'][2,ind_depth]
        t4 = f.variables['w'][3,ind_depth]
        t5 = f.variables['w'][4,ind_depth]
        t6 = f.variables['w'][5,ind_depth]
        t7 = f.variables['w'][6,ind_depth]
        t8 = f.variables['w'][7,ind_depth]
        t9 =  f.variables['w'][8,ind_depth]
        t10 = f.variables['w'][9,ind_depth]
        t11 = f.variables['w'][10,ind_depth]
        t12 = f.variables['w'][11,ind_depth]
        tracer = [t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12]

    tracershape = np.shape(tracer)

    # ==============================================================================
    # Testing if a netcdf file with the same name exists, if yes, it must be removed
    # to save a new one.

    if os.path.isfile(output_directory+netcdf_name) and delete_old_netcdf:
      os.remove(output_directory+netcdf_name)
      print "The netcdf file "+netcdf_name+" has been deleted to make room for your file of the same name."
    elif os.path.isfile(netcdf_name):
      statement = "The netcdf file "+netcdf_name+" already exists! It must be removed for a new one to be created. This can be done by changing your settings."
      sys.exit(statement)

    if not os.path.isdir(output_directory):
      os.makedirs(output_directory)
      print 'Directory '+output_directory+' has been created'

    # ==============================================================================
    # Creating netcdf file
    if save_netcdf:  
      import time
      w_nc_fid = Dataset(output_directory+netcdf_name, 'w', format='NETCDF4_CLASSIC')      # Create and open new netcdf file to write to
      w_nc_fid.description = 'current speed at surface'
      w_nc_fid.history     = 'Created ' + time.ctime(time.time())

      nod3d    = w_nc_fid.createDimension('nod2d', mesh.n2d)               # Create dimension: number of 3d nodes
      time	   = w_nc_fid.createDimension('time', tracershape[0]) 

      w_nc_var = w_nc_fid.createVariable(tracername, 'f8',('time','nod2d'))           # 'DIN' is name of saved variable
                                                                           # 'f8' sets presicion to 64-bit floating point
      w_nc_var.setncatts({'long_name': u'w at 50m',\
                          'units': u'm / s'})
      w_nc_fid.variables[tracername][:] = tracer   
      w_nc_fid.close()                                                     # close the new file                

      cwd = os.getcwd()
      print "New netcdf file (",netcdf_name,") has been created."
      print "Location: "+output_directory
    else:
      print 'You have specified not to save your field in netcdf file'

The netcdf file w50m.1985.monthly.nc has been deleted to make room for your file of the same name.
New netcdf file ( w50m.1985.monthly.nc ) has been created.
Location: /scratch/usr/hbkoziel/Arc12/netcdf/
The netcdf file w50m.1986.monthly.nc has been deleted to make room for your file of the same name.
New netcdf file ( w50m.1986.monthly.nc ) has been created.
Location: /scratch/usr/hbkoziel/Arc12/netcdf/
The netcdf file w50m.1987.monthly.nc has been deleted to make room for your file of the same name.
New netcdf file ( w50m.1987.monthly.nc ) has been created.
Location: /scratch/usr/hbkoziel/Arc12/netcdf/
The netcdf file w50m.1988.monthly.nc has been deleted to make room for your file of the same name.
New netcdf file ( w50m.1988.monthly.nc ) has been created.
Location: /scratch/usr/hbkoziel/Arc12/netcdf/
The netcdf file w50m.1989.monthly.nc has been deleted to make room for your file of the same name.
New netcdf file ( w50m.1989.monthly.nc ) has been created.
Location: /scratch/usr/hbk